In [51]:
from colmap_testing.colmap_helpers import read_write_model
import os
import cv2
import numpy as np

In [52]:
db_folder = "colmap_ws_aegis_room"
#db_folder = "colmap_ws_meeting_room"
#db_folder = "colmap_ws_meeting_room_2"

path_to_images = os.path.join("colmap_testing", db_folder,"images")
image_names = os.listdir(path_to_images)
image_names = sorted([int(os.path.splitext(name)[0]) for name in image_names])


In [53]:
path_to_model = os.path.join("colmap_testing", db_folder, "sparse","0")
#path_to_images = os.path.join("colmap_testing", "colmap_ws_aegis_room", "dense", "0", "images")
#image_names = os.listdir(path_to_images)
#image_names = sorted([int(os.path.splitext(name)[0]) for name in image_names])
image_paths = [os.path.join(path_to_images, str(name) + ".png") for name in image_names]

rgb_images = np.array([np.array(cv2.imread(imagepath)) for imagepath in image_paths])
colmap_model = read_write_model.read_model(path_to_model, ".bin")

#cameras:fx, fy, cx, cy, width, height
#images:camera_id, qvec, tvec, name 
#points:xyz, rgb, error, image_ids, point2D_idxs
cameras, images, points = read_write_model.read_model(path_to_model, ".bin")
print(path_to_model)
print(len(images))

# 3D point list with one line of data per point:
# POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)
points_keys = list(points.keys())

random_key = np.random.choice(points_keys)
random_point = points[random_key]
random_point_3d = random_point.xyz
z = random_point_3d[2]
print(random_point)

# Image list with two lines of data per image:
#   IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
#   POINTS2D[] as (X, Y, POINT3D_ID)

#first_image = images[1]

# Camera list with one line of data per camera:
#   CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]
#first_camera = cameras[1]



#The coordinates of the projection/camera center are given by -R^t * T, 
# where R^t is the inverse/transpose of the 3x3 rotation matrix composed from the quaternion and T is the translation vector. 
# The local camera coordinate system of an image is defined in a way that the X axis points to the right, 
# the Y axis to the bottom, and the Z axis to the front as seen from the image.

#calculate the projection/camera center

colmap_testing\colmap_ws_meeting_room_2\sparse\0
30
Point3D(id=5412, xyz=array([ -1.69630147,  10.31656716, 202.22393382]), rgb=array([87, 88, 91]), error=array(1.52622696), image_ids=array([17, 19, 20, 27, 28, 29, 23, 11, 12, 26, 30,  2]), point2D_idxs=array([1303,  932, 1625, 1683, 1787, 1442, 1120, 1598, 2046, 1435, 1324,
       1115]))


In [54]:

#sort keys by name in the dictionary images
def sort_keys_by_name(images):
    keys = images.keys()

    #get each name from the images
    names = []
    for key in keys:
        names.append(images[key].name)
    #remove the extension from the name
    names = [int(os.path.splitext(name)[0]) for name in names]
    sorted_idx = np.argsort(names)
    keys = np.array(list(keys))[sorted_idx]
    return keys

In [55]:
def get_frames(images, cameras, scale=1):
    frames = []
    keys = sort_keys_by_name(images)
    
    for key in keys:
        img = images[key]
        # rotation
        R = read_write_model.qvec2rotmat(img.qvec)

        # translation
        t = img.tvec

        # invert
        t = -R.T @ t
        R = R.T

        # intrinsics
        cam = cameras[img.camera_id]
        if cam.model in ("SIMPLE_PINHOLE", "SIMPLE_RADIAL", "RADIAL"):
            fx = fy = cam.params[0]
            cx = cam.params[1]
            cy = cam.params[2]
        elif cam.model in ("PINHOLE", "OPENCV", "OPENCV_FISHEYE", "FULL_OPENCV"):
            fx = cam.params[0]
            fy = cam.params[1]
            cx = cam.params[2]
            cy = cam.params[3]
        else:
            raise Exception("Camera model not supported")

        # intrinsics
        K = np.identity(3)
        K[0, 0] = fx
        K[1, 1] = fy
        K[0, 2] = cx
        K[1, 2] = cy
        frames.append((R, t, K))
    return frames

#frames are composed of R, t, K
frames = get_frames(images, cameras)

In [57]:
#project 3d point using the camera projection matrix onto the rgb image
#project them all into the same image
from matplotlib import pyplot as plt

if not os.path.exists(os.path.join("colmap_testing","reprojections",db_folder)):
    os.makedirs(os.path.join("colmap_testing","reprojections",db_folder))
subplots = []
print(len(frames))
for i,frame in enumerate(frames):
    # only show 10 out of the 30 frames
    if i%3 == 0:
        R, t, K = frame
        #add nested to t
        t = np.array([t]).T
        P = K @ np.hstack((R, t)) #projection matrix
        point_on_image = P @ np.hstack((random_point_3d, 1))
        point_on_image = point_on_image / point_on_image[2]
        print(point_on_image)
        plt.scatter(1080-point_on_image[0], 1080-point_on_image[1], c="r")
        plt.imshow(rgb_images[i])
        plt.savefig(os.path.join("colmap_testing","reprojections",db_folder,"{}.png".format(i)))
        #plt.show()
        plt.close()

    

    



30
[1.27141893e+03 7.48352489e+02 1.00000000e+00]
[1.06473662e+03 7.52689548e+02 1.00000000e+00]
[831.27850414 671.38525686   1.        ]
[675.76689988 749.99107689   1.        ]
[1.25736830e+03 6.78649866e+02 1.00000000e+00]
[671.78371524 617.10835156   1.        ]
[1.53004156e+03 9.46051503e+02 1.00000000e+00]
[550.48226412 566.98451432   1.        ]
[1.61625229e+03 9.54744916e+02 1.00000000e+00]
[854.82634482 631.11926632   1.        ]
